# Initalize libraries

## Import libraries

In [ ]:
# general
import sys, os
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
from glob import glob
from tqdm.auto import tqdm

# Data
import xarray as xr
import h5py
import numpy as np
import imageio
from nexusformat.nexus import *

# Plotting
import matplotlib.pyplot as plt

# skimage
import skimage.morphology
from skimage.draw import ellipse

# scipy
from scipy.ndimage.filters import gaussian_filter
from scipy import stats

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
import fthcore as fth
import helper_functions as helper
import mask_lib
import interactive
from interactive import cimshow
import reconstruct_rb as rec

# Facility specific loading functions
import MAXI_loading as loading

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
    import Phase_Retrieval as PhR
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
#%load_ext jupyter_black

## Experiment specific Functions

In [ ]:
PROPOSAL = 11018955
USER = getuser()

BASEFOLDER = "/asap3/petra3/gpfs/p04/2024/data/%s/" % PROPOSAL
sample_name = "2403_tomo_"
spefile_name = "2403_tomo"

# Load mnemonics for easy data handling
mnemonics = loading.mnemonics

### Loading data

In [ ]:
def load_data(scan_id, keypath = mnemonics["pre_scan_snapshot"], keys = None):
    """
    Load data of all specified keys from keypath
    
    Parameter
    =========
    scan_id : int
        experimental identifier of scan
    keypath : str
        path of nexus file tree to relevant data field
    keys : str or list of strings
        keys to load from keypath
        
    Output
    ======
    data : dict
        data dictionary of keys
    ======
    author: ck 2024
    """
        
    #Generate filename from scan_id
    fname = loading.generate_filename(join(BASEFOLDER,"raw"), sample_name, ".h5", scan_id)
    
    # load data with basic loading function
    data = loading.load_data(fname, keypath, keys = keys)
    
    return data


def load_key(scan_id, key):
    """
    Load any kind of data specified by key (path)
    
    Parameter
    =========
    scan_id : int
        experimental identifier of scan
    key : str
        key path of nexus file tree to relevant data field
   
    Output
    ======
    data : dict
        data dictionaray on single key
    ======
    author: ck 2024
    """
    #Generate filename from scan_id
    fname = loading.generate_filename(join(BASEFOLDER,"raw"), sample_name, ".h5", scan_id)
    
    # load data with basic loading function
    data = loading.load_key(fname, key)
    
    return data

def load_image(im_id, camera_type = "ccd"):
    """
    
    """
    # Get spe file ids
    if camera_type == "ccd":
        data = load_key(scan_id, mnemonics[camera_type])
        spe_id = str(data[mnemonics["ccd"]])[-9:-5]

        fname = join(BASEFOLDER, "raw", spefile_name + "  " + spe_id + ".spe")
        image = np.array(imageio.mimread(fname, memtest="5000MB"))
    elif camera_type == "cmos":
        pass

    return image

In [ ]:
# Full image loading procedure
def load_processing(im_id, crop=None):
    """
    Loads images, averaging of two individual images (scans in tango consist of two images),
    padding to square shape, Additional cropping (optional)
    """

    # Load data
    images = load_images(im_id)

    # Zeropad to get square shape
    #images = helper.padding(images)

    # Calculate mean
    if images.ndim > 2:
        image = np.mean(images, axis=0)
    else:
        image = images.copy()

    # Optional cropping
    if crop is not None:
        images = images[:, :crop, :crop]
        image = image[:crop, :crop]

    return image

### Loading, saving fth & cdi data

In [ ]:
# Saving of log files for fth and cdi recos
def save_fth_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["topo_centered"] = topo_c
    data["im_centered"] = im_c
    data["holo"] = holo
    data["recon"] = recon
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist
    data["phase"] = phase
    data["mask_bs"] = mask_pixel_smooth
    data["bs_smoothing"] = bs_smoothing
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general, "Logs", "Data_ImId_%04d_RefId_%s_%s" % (im_id, topo_id, USER)
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)


def save_cdi_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["pos"] = pos
    data["neg"] = neg
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist_cdi
    data["phase"] = phase_cdi
    data["mask_bs"] = mask_bs_cdi
    data["supportmask"] = supportmask
    data["mask_pixel"] = mask_pixel
    data["p_pc"] = p_pc
    data["n_pc"] = n_pc
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s" % (im_id, topo_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return


def save_topo_holo(topo_c, pos_id, neg_id):
    """
    Save only topo holos which can be later used for single helicity reconstructions
    """
    data = {}
    data["pos_id"] = pos_id
    data["neg_id"] = neg_id
    data["topo"] = topo_c

    filename = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s" % (pos_id, neg_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return

def load_fth(im_id, topo_id):
    """
    Load fth dataset
    """
    fname = join(
        folder_target,
        "Logs",
        "Data_ImId_%s_RefId_%s_%s.hdf5"
        % (
            im_id,
            topo_id,
            USER,
        ),
    )

    with h5py.File(fname, "r") as f:
        data = {}
        for key in f.keys():
            if key != "experimental_setup":
                data[key] = f[key][()]

    return data

def load_topo_holo(pos_id, neg_id):
    """
    Load topo holos for single helicity reconstructions
    """
    fname = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s.hdf5" % (pos_id, neg_id, USER),
    )

    with h5py.File(fname, "r") as f:
        im_out = f["topo"][()]
    return im_out


def load_cdi(im_id, topo_id):
    """
    Load cdi dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s.hdf5" % (im_id, topo_id, USER),
    )

    with h5py.File(fname, "r") as f:
        data = {}
        for key in f.keys():
            if key != "experimental_setup":
                data[key] = f[key][()]
    return data_cdi

### Other

In [ ]:
def differential_operator(shape,center, experimental_setup, angle = 0):
    '''
    Calculates Fourier-space differential operator for heraldo reconstruction
    
    Parameter
    ---------
    shape: int tuple
        shape of output array
    center: tuple
        array center coordinates (y,x) 
    experimental_setup: dict
        must contain detector pixel_size ["px_size"], distance ["ccd_dist"],
        wavelength ["lambda"]
    angle: float
        rotation angle of heraldo slit
        
    Returns:
    --------
    return: complex array
        differential operator in Fourier space
    '''
    
    # Convert deg to rad
    angle = np.deg2rad(angle)
    
    # Create x,y grid to convert pixel in q-space
    x, y = np.meshgrid(np.arange(shape[0]),np.arange(shape[0]))
    
    # Center meshgrid
    y, x = y - center[0],  x - center[1]
    
    # Multiplay with pixel size
    y, x = y*experimental_setup["px_size"], x*experimental_setup["px_size"]
    
    # Convert to q-space
    qy = 4*np.pi/experimental_setup["lambda"] * np.sin(0.5*np.arctan(y/experimental_setup["ccd_dist"]))
    qx = 4*np.pi/experimental_setup["lambda"] * np.sin(0.5*np.arctan(x/experimental_setup["ccd_dist"]))
    
    # Normalize q space to [-1,1]
    qy, qx = qy/np.max(np.abs(qy)), qx/np.max(np.abs(qx))
    
    return 2j*np.pi*qx*np.cos(angle) + 2j*np.pi*qy*np.sin(angle)

In [ ]:
def reconstruct_heraldo(holo, experimental_setup, center = None, prop_dist = 0, phase = 0,angle = 0):
    '''
    Reconstruction of holograms in heraldo reference scheme
    
    Parameter:
    ----------
    holo: array
        Centered input hologram
    experimental_setup: dict
        must contain detector pixel_size ["px_size"], distance ["ccd_dist"],
        wavelength ["lambda"]
    center: tuple or None
        array center coordinates (y,x) 
    prop_dist: float
        propagation distance
    phase: float
        global phase shift of complex array
    angle: float
        rotation angle of heraldo slit
        
    returns:
    image: array
        reconstructed image
    heraldo_operator: complex array
        differential operator in Fourier space
    '''
    
    if center is None:
        center = np.array(holo.shape)/2
    
    heraldo_operator = differential_operator(holo.shape,center, 
                    experimental_setup, angle = angle)
    holo = holo * heraldo_operator
    holo = fth.propagate(holo, prop_dist*1e-6, experimental_setup = experimental_setup)*np.exp(1j*phase)
    image = fth.reconstruct(holo)
    
    return image, heraldo_operator

In [ ]:
# Worker which performs complete fth reconstruction process
def worker(image, topo):
    # Centering
    shift_c = np.array(topo.shape) / 2 - center
    topo_c = cci.shift_image(topo, shift_c)
    im_c = cci.shift_image(image, shift_c)
    
    ## Image registration
    #shift = cci.image_registration(
    #    im_c[roi_im_reg],
    #    topo_c[roi_im_reg],
    #    method="dipy",
    #)
    shift = [0,0]
    print("Relative shift is: %s"%shift)

    
    # Correct relative drift
    topo_c = cci.shift_image(topo_c, shift) 
    
    # Optional ewalds sphere projection
    if project_ewalds_sphere is True:
        %%time
        proj_im = PhR.inv_gnomonic2(im_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)
        proj_topo = PhR.inv_gnomonic2(topo_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)

        im_c = proj_im.copy()
        topo_c = proj_topo.copy()

    # Create masks
    # Load polygon mask and shift to manually determined position
    mask = load_poly_masks(im_c.shape, polygon_names)
    mask = np.round(cci.shift_image(mask,mask_shift))
    
    # Increase/Decrease mask size
    footprint = skimage.morphology.disk(np.abs(mask_scale))
    if mask_scale > 0:
        mask = skimage.morphology.dilation(mask, footprint) # increase size
    elif mask_scale < 0: 
        mask = skimage.morphology.erosion(mask, footprint) # decrease size
    
    # BS center part
    mask_bs_center = cci.circle_mask(
        topo.shape, np.array(topo.shape) / 2, bs.rBS-3, sigma=None
    )

    # Create image specific beamstop mask
    mask_im = mask.copy()
    mask_im = mask_im + (im_c > 40000) + mask_bs_center
    mask_im[mask_im > 1] = 1

    # Create topo specific beamstop mask
    mask_topo = mask.copy()
    mask_topo = mask_topo + (topo_c > 40000) + mask_bs_center

    # Combine both
    mask_pixel = mask_im + mask_topo
    mask_pixel[mask_pixel > 1] = 1
    
    # Create smooth mask
    footprint = skimage.morphology.disk(6)
    mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
    mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 2)

    # Get scaling factor and offset
    factor, offset = cci.dyn_factor(
        im_c * (1 - mask_pixel),
        topo_c * (1 - mask_pixel),
        method="correlation",
        print_out=False,
        plot=False,
    )

    # Calculate differences (magnetic) and sums (topographc) contrast holograms.
    # _c: centered, without beamstop, _b: centered, with beamstop
    diff_c = im_c / factor - topo_c - offset
    diff_b = diff_c * mask_bs
    sum_c = im_c / factor + topo_c - offset
    sum_b = sum_c * mask_bs

    if heraldo is True:
        #holo = sum_c * heraldo_operator * mask_bs*(1 - mask_pixel_smooth)
        holo = diff_c * heraldo_operator * mask_bs*(1 - mask_pixel_smooth)
    elif heraldo is False:
        #holo = sum_c * mask_bs*(1 - mask_pixel_smooth)
        holo = diff_c * mask_bs*(1 - mask_pixel_smooth)

    # Reconstruct
    recon = cci.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )

    # worker dictionary
    worker_dict = {}
    worker_dict["center"] = center
    worker_dict["topo_c"] = topo_c
    worker_dict["im_c"] = im_c
    worker_dict["recon"] = recon
    worker_dict["factor"] = factor
    worker_dict["offset"] = offset
    worker_dict["shift"] = shift
    worker_dict["diff_c"] = diff_c
    worker_dict["sum_c"] = sum_c
    worker_dict["mask_bs"] = mask_bs
    worker_dict["mask_pixel"] = mask_pixel
    
    return worker_dict

In [ ]:
# Setup phase and propagation for cdi once
phase_cdi = 0
prop_dist_cdi = 0
dx = 0
dy = 0


def phase_retrieval(
    pos, neg, mask_pixel, supportmask, vmin = 0, Startimage=None, Startgamma=None
):
    # Prepare Input holograms
    pos2 = pos.copy()
    neg2 = neg.copy()

    mi, _ = np.percentile(pos2[pos2 != 0], [vmin, 99.9])
    pos2 = pos2 - mi
    mi, _ = np.percentile(neg2[neg2 != 0], [vmin, 99.9])
    neg2 = neg2 - mi

    pos2[pos2 < 0] = 0
    neg2[neg2 < 0] = 0
    pos2 = pos2.astype(complex)
    neg2 = neg2.astype(complex)

    bsmask_p = mask_pixel.copy()
    bsmask_p[pos2 <= 0] = 1
    bsmask_n = mask_pixel.copy()
    bsmask_n[neg2 <= 0] = 1

    # Setup start image and startgamma
    if Startimage is None:
        Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask)))
    else:
        Startimage = Startimage.copy()
    if Startgamma is None:
        Startgamma = np.ones(pos.shape) * 1e-6 * 2
        Startgamma[pos.shape[0] // 2, pos.shape[1] // 2] = 0.7
    else:
        Startgamma = Startgamma.copy()

    # Settings for phase retrieval reconstructions
    partial_coherence = True

    # Setup
    retrieved_p = np.zeros(pos2.shape, np.cdouble)
    retrieved_n = np.zeros(pos2.shape, np.cdouble)

    # Algorithms and Inital guess
    plt.rcParams["figure.dpi"] = 100
    print("CDI - larger mask")

    algorithm_list = ["mine", "mine", "mine"]
    Nit_list = [700, 50, 50]  # iterations for algorithm_list

    x = (np.sqrt(np.maximum(pos2, np.zeros(pos2.shape)))[mask_pixel == 0]).flatten()
    y = ((np.abs(Startimage))[mask_pixel == 0]).flatten()
    res = stats.linregress(x, y)
    Startimage -= res.intercept
    Startimage /= res.slope

    average_img = 30
    real_object = False  # always set to False

    if partial_coherence:
        RL_freq = 20
        RL_it = 50

        algorithm_list_pc = ["mine", "ER", "ER"]
        Nit_list_pc = [700, 50, 50]

    # Execute Phase retrieval
    start_time = time.time()
    for i in range(len(Nit_list) // 3):
        print("############ -   CDI")

        # Positive helicity - beta_mode="arctan"
        retrieved_p, Error_diff_p, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i],
            beta_zero=0.5,
            Nit=Nit_list[3 * i],
            beta_mode="arctan",
            plot_every=349,
            Phase=Startimage,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Positive helicity - beta_mode="const"
        retrieved_p, Error_diff_p2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 1],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 1],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Negative helicity - beta_mode="arctan"
        retrieved_n, Error_diff_n2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(neg2, np.zeros(neg2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 2],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 2],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p * np.sqrt(np.sum(neg2) / np.sum(pos2)),
            seed=False,
            real_object=real_object,
            bsmask=bsmask_n,
            average_img=average_img,
            Fourier_last=True,
        )

        print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

        Startimage = retrieved_p.copy()

        # Partial coherence phase retrieval
        if partial_coherence:
            # CDI_PC
            print("############   -   CDI_pc")
            pos3 = (np.abs(retrieved_p) ** 2) * bsmask_p + np.maximum(
                pos2, np.zeros(pos2.shape)
            ) * (1 - bsmask_p)
            neg3 = (np.abs(retrieved_n) ** 2) * bsmask_n + np.maximum(
                neg2, np.zeros(neg2.shape)
            ) * (1 - bsmask_n)

            # retrieve pos image
            (
                retrieved_p_pc,
                Error_diff_p_pc,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list_pc[3 * i],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i],
                beta_mode="arctan",
                gamma=Startgamma,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=349,
                Phase=Startimage,
                seed=False,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            (
                retrieved_p_pc,
                Error_diff_p_pc2,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 1],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 1],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )
            (
                retrieved_n_pc,
                Error_diff_n_pc2,
                Error_supp,
                gamma_n,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(neg3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 2],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 2],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc * np.sqrt(np.sum(neg2) / np.sum(pos2)),
                real_object=False,
                bsmask=np.zeros(bsmask_n.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

            Startimage = retrieved_p_pc.copy()
            Startgamma = gamma_p.copy()

    print("Phase Retrieval Done!")

    return (
        retrieved_p,
        retrieved_n,
        retrieved_p_pc,
        retrieved_n_pc,
        bsmask_p,
        bsmask_n,
        gamma_p,
        gamma_n,
    )

# Experimental Details

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.20,  # ccd to sample distance
    "px_size": 13.5e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
    "oversaturation" : 60e3, # Pixel saturation threshold
    "energy": 778
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)
experimental_setup["z_angle_offset"] = 67

# General saving folder and log folder
folder_general = helper.create_folder(join(BASEFOLDER, "processed"))
helper.create_folder(join(folder_general, "Logs"))
helper.create_folder(join(folder_general, "Topos"))
print("Output Folder: %s" % folder_general)

# Load images

Start by loading the images: image of interest (im), reference of charge scattering (topo), any kind of dark image (dark)

We estalished the following convention: Difference Hologram which contains only the magnetic scattering will be calculated according to:

$Diff = \frac{Image}{factor} - Topo$,

where the factor is used for intensity scaling. In Case that you recorded scans of the same magnetic state with both helicities, use the image with negative helicity as topo and the one with positive helicity as image

In [ ]:
# Define scan ids for each image
im_id = 1416 # single helicity mode: image with magnetic contrast, double helicity: pos
dark_id = 1417
topo_id = (
    1418  # single helicity mode: image without magnetic contrast, double helicity: neg
)

# Are you using a HERALDO dataset?
heraldo = False

# Are you going to use ewalds sphere projection?
project_ewalds_sphere = False

# Load energy and add to experimental setup
experimental_setup["energy"] = load_pre_scan_snapshot(im_id,"energy")
experimental_setup["lambda"] = helper.photon_energy_wavelength(
    experimental_setup["energy"], input_unit="eV"
)

# Load angles
experimental_setup["srotx"] = load_pre_scan_snapshot(im_id,"srotx")
experimental_setup["srotz"] = load_pre_scan_snapshot(im_id,"srotz")
experimental_setup["angle"] = experimental_setup["srotz"] - experimental_setup["z_angle_offset"]

print("Image Id: %s" % im_id)
print("Topo Id: %s" % topo_id)
print("Dark Id: %s" % dark_id)
print("Sample rotation angle: %.2f"%experimental_setup["angle"])



## Load image of interest

In [ ]:
# Load image
image = load_processing(im_id, crop=None)

# Plot
fig, ax = cimshow(image)
ax.set_title("Image")

## Load topo data set and average

In [ ]:
# Load topo
topo = load_processing(topo_id , crop=None)

# Plot
fig, ax = cimshow(topo)
ax.set_title("Topo")

## Load dark image

In [ ]:
# Load image
if dark_id is not None:
    dark = load_processing(dark_id, crop=None)
    
    image = image - dark
    topo = topo - dark
    
    # Plot
    fig, ax = cimshow(dark)
    ax.set_title("Dark")

In [ ]:
# Plot
fig, ax = cimshow(image)
ax.set_title("Image")

# Center holograms

* Find center of the hologram to get a well-defined q-space. 
* Create smooth mask for beamstop or overexposed areas in direct beam

## Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. Circles are just a guide to eye and will not be used otherwise.

In [ ]:
# Find center position via widget
c0, c1 = [516, 509]  # initial values
ic = interactive.InteractiveCenter(image, c0=c0, c1=c1)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

## Azimuthal integrator widget for finetuning
More of an "expert widget" which works very well for alignment if you have an Airy Pattern as a scattering image. PyFai transforms images from carthesian detector coordinate system into polar coordinate system with angle `phi` and radial distance `q` as axis (azimuthal transformation). The center of the coordinate system will be defined in the azimuthal integrator class and must not necessarily represents the center coordinates of your image array. If the center is set correctly, all rings of the Airy pattern will be transformed into a straight line in the I(q,chi)-plot as rings appear at a given q for all angles chi.

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Not the widget, just for double checking to find correct radial range for plotting
# Perform azimuthal transformation
I_t, q_t, phi_t = ai.integrate2d(
    np.log10(image-np.min(image)+1),
    500,  # number of points for phi
    radial_range=(0.025, 0.1),  # relevant q-range
    unit="q_nm^-1",
    correctSolidAngle=False,
    method = "BBox"
)
# Combine in an xarray for plotting
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.percentile(I_t, [1, 95])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

In [ ]:
# The widget
aic = interactive.AzimuthalIntegrationCenter(
    #np.log10(image-np.min(image)+1),
    image,
    ai,
    c0=center[0],
    c1=center[1],
    im_data_range=[1, 98],
    radial_range=(0.025, 0.05),
    qlines=[100, 110],
)

In [ ]:
# Get center positions from widget
center = [aic.c0, aic.c1]
print(f"Center:", center)

## Centering of image hologram

In [ ]:
# Apply to topo and image
shift_c = np.array(image.shape) / 2 - center
im_c = cci.shift_image(image, shift_c)
topo_c = cci.shift_image(topo, shift_c)  # centered image

# Image Registration

Relative drift between data holograms and their corresponding topo holograms is calculated by image registration algorithm. Necessary to get well defined difference hologram. The reference is always the static background image (topo).

## Set Alignment ROI 

Set a region of interest (ROI) of reference (topo) use for image registration is performed. Can include beamstop when beamstop mask was defined.

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(im_c)
ax.set_title("Don't include the beamstop as this will misdirect the algorithm")

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi_im_reg = np.array([y1, y2, x1, x2]).astype(int)
roi_im_reg_s = np.s_[roi_im_reg[0] : roi_im_reg[1], roi_im_reg[2] : roi_im_reg[3]]

print(f"Image registration roi:", roi_im_reg)

## Calculate drift of images

In [ ]:
shift = cci.image_registration(
    im_c[roi_im_reg_s],
    topo_c[roi_im_reg_s],
    method="dipy",
)
print("Relative shift is: %s"%shift)

In [ ]:
# Define shift manually for comparison
tmp_shift = [0.03, 0.02]

# Loop over shifts
temp_diff = np.zeros((3, im_c.shape[0], im_c.shape[1]))
shifts = [
    [0, 0],
    tmp_shift,
    -shift,
]
for i, tshift in enumerate(shifts):
    temp = cci.shift_image(im_c, tshift)
    temp_factor = cci.dyn_factor(temp, topo_c)
    temp_diff[i] = temp - temp_factor[0] * topo_c

# Plots for comparision
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
mi, ma = np.percentile(temp_diff[0], [.1, 99.9])
ax[0].imshow(temp_diff[0], vmin=mi, vmax=ma)
ax[0].set_title("Zero shift")
mi, ma = np.percentile(temp_diff[1], [.1, 99.9])
ax[1].imshow(temp_diff[1], vmin=mi, vmax=ma)
ax[1].set_title("Manual shift: %s" % shifts[1])
mi, ma = np.percentile(temp_diff[2], [.1, 99.9])
ax[2].imshow(temp_diff[2], vmin=mi, vmax=ma)
ax[2].set_title("Auto shift: %s" % np.round(shifts[2], 2))

## Correct drift of topo

In [ ]:
# Correct relative drift
topo_c = cci.shift_image(topo_c, shift)  

# Plot original and shifted holos
mi, ma = np.percentile(np.real(im_c[im_c != 0]), (0.1, 99))
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 4))
ax[0].imshow(np.real(topo), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Uncentered topo")
ax[1].imshow(np.real(topo_c), cmap="viridis", vmin=mi, vmax=ma)
ax[1].set_title("Centered topo with beamstop")

# Add circles with different radi r
tmp = np.array(image.shape) / 2
for r in np.arange(50, 200, 25):
    ax[0].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))
    ax[1].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))

# Calculate Projection

In [ ]:
if project_ewalds_sphere is True:
    %%time
    proj_im = PhR.inv_gnomonic2(im_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)
    proj_topo = PhR.inv_gnomonic2(topo_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)
    
    im_c = proj_im.copy()
    topo_c = proj_topo.copy()

# Create beamstops

We want to cover the beamstop with a smooth circle to cover its sharp edges as these would create ringing-like artifacts in the reconstruction plane. Make it only as large as necessary to keep as much information as possible.

## Manual masking of beamstop wires

Just mask the beamstop wires, broken pixels, etc. 

In [ ]:
poly_mask = interactive.draw_polygon_mask(im_c)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.get_vertice_coordinates()
mask_draw = poly_mask.full_mask.astype(int)

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(im_c * mask_draw, [0.1, 99.9])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")
plt.tight_layout()

In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Mask #1
    mask_coordinates["test"] = [[(508.4, 501.7), (505.9, 503.4), (502.5, 507.4), (501.9, 510.3), (502.1, 515.9), (502.7, 518.5), (506.0, 522.4), (509.1, 524.3), (512.2, 525.4), (515.6, 524.7), (517.8, 523.6), (521.5, 521.0), (523.6, 518.0), (524.0, 515.9), (523.9, 508.1), (519.4, 502.6), (514.9, 500.5), (510.3, 500.7)]]
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load? You can combine multiple masks from
# load_poly_coordinates(). Just add names of mask as strings to list like
# ["bs_small","bs_medium"]
polygon_names = ["test"]  # ["bs_test"]
mask_draw = mask_lib.load_poly_masks(
    experimental_setup["binning"] * np.array(image.shape),
    load_poly_coordinates(),
    polygon_names,
)
#mask_draw = np.zeros(im_c.shape)

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

#mi, ma = np.percentile(im_c * mask_draw, [0.1, 90])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")

## Finetuning of mask position

In [ ]:
# Use widget to shift and expand or shrink the mask
ss_mask = interactive.Shift_Scale_Mask(im_c,mask_draw)

In [ ]:
# Take mask, shift and scaling from widget
mask_draw, mask_shift, mask_scale = ss_mask.get_mask()

## Overview beamstops
Verify good beamstop alignment

In [ ]:
# Smoothing of beamstop mask
bs_smoothing = 2

# Add circular beamstop mask
mask_im = mask_draw.copy()
mask_topo = mask_draw.copy()

# Mask over-saturated pixel
mask_im = mask_im + (im_c > experimental_setup["oversaturation"])
mask_topo = mask_topo + (topo_c > experimental_setup["oversaturation"])

# Combine both
mask_pixel = mask_im + mask_topo
mask_pixel[mask_pixel > 1] = 1

# Create smooth mask for FTH reconstructions
footprint = skimage.morphology.disk(3*bs_smoothing)
mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, smoothing_bs)

# Plot both
fig, ax = plt.subplots(2, 4, figsize=(10, 5), sharex=True, sharey=True)
mi, ma = np.percentile(im_c, [1, 99.9])
ax[0, 0].imshow(im_c, vmin=mi, vmax=ma)
ax[0, 0].set_title("Image")
mi, ma = np.percentile(im_c * mask_im, [1, 99.99])
ax[0, 1].imshow(im_c * mask_im, vmin=mi, vmax=ma)
ax[0, 1].set_title("Image*mask")
mi, ma = np.percentile(im_c * (1 - mask_im), [0.1, 99.9])
ax[0, 2].imshow(im_c * (1 - mask_im), vmin=mi, vmax=ma)
ax[0, 2].set_title("Image*(1-mask)")
ax[0, 3].imshow(mask_pixel_smooth)
ax[0, 3].set_title("Combined Mask")

mi, ma = np.percentile(topo_c, [1, 99.9])
ax[1, 0].imshow(topo_c, vmin=mi, vmax=ma)
ax[1, 0].set_title("Topo")
mi, ma = np.percentile(topo_c * mask_im, [1, 99.99])
ax[1, 1].imshow(topo_c * mask_topo, vmin=mi, vmax=ma)
ax[1, 1].set_title("Topo*mask")
mi, ma = np.percentile(topo_c * (1 - mask_topo), [0.1, 99.9])
ax[1, 2].imshow(topo_c * (1 - mask_topo), vmin=mi, vmax=ma)
ax[1, 2].set_title("topo*(1-mask)")
mi, ma = np.percentile((im_c - topo_c) * (1 - mask_pixel_smooth), [0.1, 99.9])
ax[1, 3].imshow((im_c - topo_c) * (1 - mask_pixel_smooth), vmin=mi, vmax=ma)
ax[1, 3].set_title("Image-Topo")

# Calculate difference holograms

You can see the reconstrution of the magnetization only after subtracting the large background that you get from the diffraction on the circular object aperture (Airy Pattern). This might require a scaling factor to correct intensity changes between the hologram and the topo. Scaling factor will be determined automatically by a linear fit. If the fit seems off, there might be an issue with the data

In [ ]:
# Get scaling factor and offset
factor, offset = cci.dyn_factor(
    im_c * (1 - mask_pixel),
    topo_c * (1 - mask_pixel),
    method="correlation",
    print_out=True,
    plot=True,
)

# Calculate differences (magnetic) and sums (topographc) contrast holograms.
# _c: centered, without beamstop, _b: centered, with beamstop
diff_c = im_c / factor - topo_c - offset
sum_c = im_c / factor + topo_c - offset

In [ ]:
# Plot an example of the difference or sum hologram
fig, ax = cimshow(diff_c * (1 - mask_pixel_smooth))
ax.set_title(f" Diff Id %d - %d" % (im_id, topo_id))

# fig, ax = cimshow(sum_b)
# ax.set_title(f" Sum Id %d" % im_id)

# Reconstruct Diff Holos (FTH)

Reconstruct the hologram.

0. If you are doing heraldo, determine the rotation angle of the hologram
1. Choose a region of interest (ROI) which means selecting one reconstruction from the reconstruction plane.
2. Propagate the image and shift the phase for maximal contrast and sharpness in your ROI

In [ ]:
# This is only relevant if you are doing heraldo
if heraldo is True:
    xticks, yticks = [495,522], [490,517]
    shift_rotate_widget = interactive.Shift_Rotate(im_c, shift = [0,0], angle = 0, ticks = (yticks,xticks))

## Set Patterson Map ROI

Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
# Choose contrast mode
# diff_c: magnetic contrast only
# sum_c: topographic contrast only
holo = diff_c* (1 - mask_pixel_smooth)
#holo = sum_c *(1 - mask_pixel_smooth)

if heraldo is True:
    print("Doing Heraldo")
    _, _, heraldo_rotation = shift_rotate_widget.get_parameter()
    tmp, heraldo_operator  = reconstruct_heraldo(holo, experimental_setup, 
                              center = None, prop_dist = 0, phase = 0,angle = heraldo_rotation)
else:
    tmp = fth.reconstruct(holo)
    
fig, ax = cimshow(np.real(tmp), cmap="gray")

In [ ]:
# Execute to get roi
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi = np.array([y1, y2, x1, x2]).astype(int)  # ystart, ystop, xstart, xstop
roi_s = np.s_[roi[0] : roi[1], roi[2] : roi[3]]
print(f"Roi Reco:{roi}")

## Tune propagation and phase
Focus the image by tuning the propagation distance. This really works like focussing in a microscope.
Phase slider will move contrast between real and imaginary part. Usually we use the phase which maximizes the contrast in the real part.

In [ ]:
# Widget
if heraldo is True:
    #holo = sum_c * heraldo_operator * (1 - mask_pixel_smooth)
    holo = diff_c * heraldo_operator * (1 - mask_pixel_smooth)
elif heraldo is False:
    #holo = sum_c *(1 - mask_pixel_smooth)
    holo = diff_c *(1 - mask_pixel_smooth)

slider_prop, slider_phase, button = fth.propagate(
    holo,
    roi_s,
    phase = 0, #Initial value
    prop_dist = 0, #Initial value
    experimental_setup=experimental_setup,
    scale=(1, 99),
)

In [ ]:
# Read prop dist and phase from widget
prop_dist = slider_prop.value
phase = slider_phase.value

print(f"Propagation distance: %0.2f" % prop_dist)
print(f"Phase: %0.2f" % phase)

## Save reconstruction

Save png files of the images and a h5 file containing all important variables

In [ ]:
# Style of reconstruction plot
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
def get_title(data_key,im_id,topo_id,CDI=False):
    # Rotation in title
    #values = np.mean(np.array(load_pre_scan_snapshot(im_id, data_key)) * 1000)
    #values = [np.round(values, 2)]
    
    # Rotation in title
    values = load_pre_scan_snapshot(im_id, data_key)

    if CDI is False:
        mode = "FTH"
    elif CDI is True:
        mode = "CDI"
        
    if data_key == "srotz":
        title = "Image %d - %d %s: %s = %.1f (%.1f deg)" % (im_id, topo_id, mode, data_key, values, values-experimental_setup["z_angle_offset"])
    else:
        title = "Image %d - %d %s: %s = %.2f "%(im_id, topo_id, mode, data_key, values)
        
    return title

In [ ]:
# Create plot
holo = diff_c *(1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )

# Create plot
title = get_title("srotz",im_id,topo_id)
plot_recon(recon[roi_s], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%04d_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save hdf5 file
save_fth_h5()

In [ ]:
# Closes all existing plots
plt.close("all")

# Batch processing FTH (update!)

# CDI Reconstruction

## Create set of pos and neg helicity holograms

CDI algorithm needs holograms recorded wih both helicity ($\sigma = \pm 1$) as input. We use will calculate those from our previously centered and intensity normalized holograms according to;´; 

$Image(\sigma) = Topo + \sigma \cdot diff $,

In [ ]:
# Copy values from FTH reco (here topo = sum_c)
pos = (sum_c + diff_c) / 2
neg = (sum_c - diff_c) / 2

## Create Support mask
The support mask is the real-space constraint used for the (holographically-aided) phase retrieval, i.e., certain details about our sample like the sample geometry. For our samples we can directly derive a very strong constraint: The FTH reconstructions show us previsely the actual real-space sample structure, i.e., the arrangement of our aperture where x-rays are transmitted ("1") while the masked areas show no transmission ("0"). We will therefore create a binary mask that reflects this transmission as an input for the phase retrieval.

How to draw a support mask: Create a binary mask of the locations of sample apertures in the fth reconstruction. Areas with apertures are "1". Select only a single set of reconstructions (object & reference apertures) that originate from a single reference. Use the widget!

### Option 1: Execute if you want to create a new support mask with elliptical mask widget

If you really want to create a new support mask, execute next cell and then the "InteractiveCircleCoordinates"-widget

In [ ]:
# How many references do you have?
nr_ref = 3

# Setup coordinates (nr_ref + 1 coordinates, as there is always the object aperture)
support_coordinates = [
    [pos.shape[-2] // 2, pos.shape[-1] // 2, 7] for k in range(nr_ref + 1)
]

# Widget to find the positions and sizes of the different apertures
print("Cover the object & reference apertures for each set of reconstructions that originates from the same reference with circles.")
print("Optimization: Change one circle parameter, calc phase retrieval image, compare with images reconstructed with old circle parameter. Repeat!")

# Create plot
#holo = diff_c * (1 - mask_pixel_smooth)
holo = pos * (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.abs(recon)
    
ds = interactive.InteractiveCircleCoordinates(recon,
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds.get_params()

# Create supportmask from coordinates
supportmask = mask_lib.create_circle_supportmask(support_coordinates, pos.shape)

# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(recon, (1, 99))
ax.imshow(recon, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.4, cmap="binary")
ax.set_title("Image with overlayed mask")

### Option 2: Execute if you want to load an existing support mask created with elliptical mask widget

In [ ]:
def get_supportmask_coordinates(sample):
    """
    Dictionary that stores coordinates of circular support mask apertures
    """

    # Setup dictonary
    support_coord = dict()

    # coordinates
    support_coord["FBEb21_Co"] = [((873.5, 952.5), 84.5, 84.5, 0.0), ((828.1, 1022.0), 10.0, 10.0, 0.0), ((848.4, 1040.0), 13.0, 13.0, 0.0), ((886.0, 1034.5), 10.0, 10.0, 0.0), ((911.0, 1028.0), 12.0, 12.0, 0.0), ((1015.1, 864.8), 10.0, 10.0, 0.0), ((1024.0, 1024.0), 10.0, 10.0, 0.0)]
    support_coord["FBEb21_Co_2x2"] = [((361.7, 440.5), 86.0, 86.0, 0.0), ((316.2, 509.5), 12.0, 12.0, 0.0), ((337.1, 527.4), 14.0, 14.0, 0.0), ((373.9, 522.5), 11.0, 11.0, 0.0), ((398.4, 515.8), 14.0, 14.0, 0.0), ((503.2, 352.5), 11.0, 11.0, 0.0), ((512.0, 512.0), 12.0, 12.0, 0.0)]
    
    return support_coord[sample]

In [ ]:
# Which supportmask to load? ("s2306a-C1", "s2308a-B1", ...)
sample = "FBEb21_Co_2x2"

# Get coordinates and create supportmask
support_coordinates = get_supportmask_coordinates(sample)

In [ ]:
# Widget to find the positions and sizes of the different apertures
print("Cover the object & reference apertures for each set of reconstructions that originates from the same reference with circles.")
print("Optimization: Change one circle parameter, calc phase retrieval image, compare with images reconstructed with old circle parameter. Repeat!")

# Plotting image
holo = sum_c* (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.abs(recon)

ds_ellipse = interactive.InteractiveEllipseCoordinates(recon,
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds_ellipse.get_params()

# Create supportmask from coordinates
supportmask = mask_lib.create_ellipse_supportmask(support_coordinates, pos.shape)

# Plotting image
holo = pos * (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    # heraldo slit
    supportmask[508:516,512:582] = 1 #horz
    
    # Do reco
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.real(recon)


# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(recon, (1, 99))
ax.imshow(recon, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.4, cmap="binary")
ax.set_title("Image with overlayed mask")

### Take Roi
Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(supportmask.astype(int))

In [ ]:
roi_cdi = interactive.axis_to_roi(ax)

## Do Phase Retrieval

In [ ]:
# Executes the algorithm
offset_vmin = .5
recon = fth.reconstruct(
    fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
    * np.exp(1j * phase)
)
Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask*recon)))
Startimage = None
Startgamma = None

(
    retrieved_p,
    retrieved_n,
    retrieved_p_pc,
    retrieved_n_pc,
    bsmask_p,
    bsmask_n,
    gamma_p,
    gamma_n,
) = phase_retrieval(pos, neg, mask_pixel.astype(int), supportmask, vmin = offset_vmin, Startimage=Startimage, Startgamma=Startgamma)

## Reconstruct images from phase retrieval

In [ ]:
# New beamstop for CDI recos as phase retrieval of low-q might be insufficient. If phase retrieval worked well
# Try without beamstop: `use_bs = False`
use_bs = False
bs_diam_cdi = 25  # diameter of beamstop


# Create beamstop
if use_bs is True:
    mask_bs_cdi = 1 - cci.circle_mask(
        topo.shape, np.array(topo.shape) / 2, bs_diam_cdi, sigma=4
    )
    mask_bs_cdi = 1 - mask_pixel_smooth.copy()
elif use_bs is False:
    mask_bs_cdi = np.ones(pos.shape)  # if you don't want a beamstop

# Get Recos partial coherence
# Positiv partial coherence
p_pc = cci.reconstruct(
    fth.propagate(
        retrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative partial coherence
n_pc = cci.reconstruct(
    fth.propagate(
        retrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Plotting
mode = "-"
print("Fine-tuning of reconstruction parameter:")
slider_prop, slider_phase, slider_dx, slider_dy = rec.focusCDI(
    retrieved_p_pc * mask_bs_cdi,
    retrieved_n_pc * mask_bs_cdi,
    #retrieved_p * mask_bs_cdi,
    #retrieved_n * mask_bs_cdi,
    roi_cdi,
    mask=supportmask,
    phase=phase_cdi,
    dx=dx,
    dy=dy,
    prop_dist=prop_dist_cdi,
    experimental_setup=experimental_setup,
    operation=mode,
    max_prop_dist=20,
    scale=(1, 99),
)

In [ ]:
# Get phase from slider
phase_cdi = slider_phase.value
prop_dist_cdi = slider_prop.value

# Reconstruct images with new parameter
p_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
) * np.exp(1j * phase_cdi)

n_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
) * np.exp(1j * phase_cdi)


print("Phase CDI: %s" % phase_cdi)
print("Prop_dist: %s" % prop_dist_cdi)

In [ ]:
# Confirm that offset subtraction in cdi function works, i.e., only small fraction of hologram is actually masked
fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
tmp = np.abs(retrieved_p_pc*mask_bs_cdi)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 0].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 0].set_title("Pos holo")

tmp = np.abs(retrieved_n_pc)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 1].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 1].set_title("Neg holo")
ax[1, 0].imshow(bsmask_p)
ax[1, 0].set_title("Pos holo mask")
ax[1, 1].imshow(bsmask_n)
ax[1, 1].set_title("Neg holo mask")

## Save reconstructions

In [ ]:
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (.1, 99.9))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (.1, 99.9))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
# Saves only real and imaginary part
#recon = p - n
recon = (p_pc - n_pc)#   / (p_pc + n_pc)

# Plot
title = get_title("srotz", im_id,topo_id, CDI=True)
plot_recon(recon[roi_cdi], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%s_cdi_diff_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save h5
save_cdi_h5()

# Batch processing CDI (needs to be updated)

## Define Scan Ids

In [ ]:
# Load support mask of which sample?
sample = "SWA-B31"

In [ ]:
# Define the sets for reconstructions. You can make a list or use np.arange
# im_id_set should always have ids of positive helicity holograms,
# topo_id_set those of negative helicity or a hologram of a saturated state

# You can also use nested lists: 
# in case topo_id_set = [[id1,id2],id3 it will use the sum hologram 
# calculated from [id1,id2] as topo

im_id_set = [1755,1763,1774]
dark_id_set = [1756,1762,1775]
topo_id_set = [1757,1761,1776]

# In case of single helicity reconstructions, adapt the helicity
# for contrast inversion
helicity = 1 * np.ones(len(im_id_set), dtype=int)  # [1,-1]

print("Dynamics Set:  %s" % im_id_set)
print("Reference Set: %s" % topo_id_set)
print("Helicity: %s" % helicity)

## Execute Phase Retrieval

In [ ]:
# Ugly Automatic processing of image stacks
recons_name = []  # for gifs
for it, im_id in enumerate(tqdm(im_id_set)):
    # Load energy and add to experimental setup
    experimental_setup["energy"] = load_pre_scan_snapshot(im_id,"energy")
    experimental_setup["lambda"] = cci.photon_energy_wavelength(
        experimental_setup["energy"], input_unit="eV"
    )
    
    # Load angles
    experimental_setup["srotx"] = load_pre_scan_snapshot(im_id,"srotx")
    experimental_setup["srotz"] = load_pre_scan_snapshot(im_id,"srotz")
    experimental_setup["angle"] = experimental_setup["srotz"] - experimental_setup["z_angle_offset"]
    
    # Load images
    image = load_processing(im_id)

    # Load dark
    dark_id = dark_id_set[it]
    if dark_id is not None:
        dark = load_processing(dark_id, crop=None)
        image = image - dark

    # Get topo
    topo_id = topo_id_set[it]

    # Construct topo holo from two helicity images
    if isinstance(topo_id, list):
        print("Using Topo from sum of two helicity holograms")
        pos_id = topo_id[0]
        neg_id = topo_id[1]

        try:
            topo = load_topo_holo(pos_id, neg_id) / 2
        except:
            topo = load_topo_holo(neg_id, pos_id) / 2
        topo = cci.shift_image(
            topo, -shift_c
        )  # shift out of center so you don't need to change the worker

        # Process images
        worker_dict = worker(image, topo)

    # usual case, single topo holo
    else:
        # Load data
        print(f"Loading imageId: %04d, topoId: %04d" % (im_id, topo_id))
        topo = load_processing(topo_id)

        if dark_id is not None:
            topo = topo - dark

        # Process images
        worker_dict = worker(image, topo)

        # Save topo hologram
        save_topo_holo(worker_dict["sum_c"], im_id, topo_id)   

    
    # Reconstruct
    recon = worker_dict["recon"]
    
    # Plot
    title = get_title("magOOP", im_id, topo_id, CDI=False)
    plot_recon(recon[roi_cdi], title)

    # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_RefId_%s_fth_diff_stack_%s.png" % (im_id, topo_id, USER),
    )
    print("Saving: %s" % fname)
    plt.savefig(fname, bbox_inches="tight", transparent=False)

    # Optional: Save hdf5 file of fth data
    save_fth_h5()

    ################ CDI ###############
    # Create pos and neg helicity set
    pos = (worker_dict["sum_c"] + worker_dict["diff_c"])/2
    neg = (worker_dict["sum_c"] - worker_dict["diff_c"])/2

    # Create mask pixel
    mask_im = worker_dict["mask_pixel"]
    mask_topo = mask_im.copy()

    # Mask over-saturated pixel
    mask_im = mask_im + (pos > 40000)
    mask_topo = mask_topo + (neg > 40000)

    # Combine both
    mask_pixel = mask_im + mask_topo
    mask_pixel[mask_pixel > 1] = 1

    # Create supportmask from coordinates
    supportmask = create_ellipse_supportmask(support_coordinates, pos.shape)

    # Do phase retrieval
    (
        retrieved_p,
        retrieved_n,
        retrieved_p_pc,
        retrieved_n_pc,
        bsmask_p,
        bsmask_n,
        gamma_p,
        gamma_n,
    ) = phase_retrieval(pos, neg, mask_pixel, supportmask, vmin = offset_vmin, Startimage=Startimage, Startgamma=Startgamma)

    # Get Recos partial coherence
    # Positiv partial coherence
    p_pc = fth.reconstructCDI(
        fth.propagate(
            retrieved_p_pc * mask_bs_cdi,
            prop_dist_cdi * 1e-6,
            experimental_setup=experimental_setup,
        )
    )
    # Negative partial coherence
    n_pc = fth.reconstructCDI(
        fth.propagate(
            retrieved_n_pc * mask_bs_cdi,
            prop_dist_cdi * 1e-6,
            experimental_setup=experimental_setup,
        )
    )

    ##### Calc reco and optimze contrast
    recon = helicity[it] * (p_pc - n_pc)
    _, phase_cdi = optimize_phase_contrast(recon, supportmask, method="contrast")
    # phase_cdi = 0
    recon = recon * np.exp(1j * phase_cdi)
    print("Phase is:", np.round(phase_cdi, 2))
    ########

    # Plot
    title = get_title("srotz",im_id,topo_id,CDI=True)
    plot_recon(recon[roi_cdi], title)

    # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_RefId_%s_cdi_stack_%s.png" % (im_id, topo_id, USER),
    )

    print("Saving: %s" % fname)
    plt.savefig(fname, bbox_inches="tight", transparent=False)
    recons_name.append(fname)

    # Save files as h5
    save_cdi_h5()

    print(" ")
print("CDI stack processing finished")